In [ ]:
!ssh-keygen -t ed25519 -C "frank.scagluso@bc.edu"

Generating public/private ed25519 key pair.
Enter file in which to save the key (/root/.ssh/id_ed25519): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_ed25519
Your public key has been saved in /root/.ssh/id_ed25519.pub
The key fingerprint is:
SHA256:rVUeJZU0mXL+wksYOBmWLU7aR7tos4BAgvp+9ZvbOSA frank.scagluso@bc.edu
The key's randomart image is:
+--[ED25519 256]--+
| .         o.o=+ |
|. . .     * +o=. |
|.  o     * *o=   |
|.   .   ..*o+..  |
| .   . .S o+.= . |
|  .   E oo+ o + .|
| .   . o.+ o . o |
|  . .   .oo.  .  |
|   .    +oo.     |
+----[SHA256]-----+


In [ ]:
!cat ~/.ssh/id_ed25519.pub

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIH2nYVrC+4iLMvhdbRDbgz7Dqmzfk+qfiooETkLYeMt3 frank.scagluso@bc.edu


In [ ]:
!ssh-keyscan github.com >> ~/.ssh/known_hosts

# github.com:22 SSH-2.0-93587f60
# github.com:22 SSH-2.0-93587f60
# github.com:22 SSH-2.0-93587f60
# github.com:22 SSH-2.0-93587f60
# github.com:22 SSH-2.0-93587f60


In [ ]:
!ssh -T git@github.com

Hi fscagz! You've successfully authenticated, but GitHub does not provide shell access.


In [ ]:
!git clone git@github.com:howellbe02/NLP-Project-Group-3.git

Cloning into 'NLP-Project-Group-3'...
remote: Enumerating objects: 1340, done.
remote: Counting objects: 100% (1340/1340), done.
remote: Compressing objects: 100% (934/934), done.
remote: Total 1340 (delta 647), reused 1083 (delta 401), pack-reused 0 (from 0)
Receiving objects: 100% (1340/1340), 12.18 MiB | 22.75 MiB/s, done.
Resolving deltas: 100% (647/647), done.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install datasets transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [ ]:
!git clone https://github.com/howellbe02/NLP-Project-Group-3.git

fatal: repository 'github.com/howellbe02/NLP-Project-Group-3.git' does not exist


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

API Key for wandb:
692f8e561cbd80fda1432e206bad1d7c23897a4d

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import math


# DEVICE SETUP
######################
#CHANGE TO COMMENTED LINE POR FAVOR (also delete these ass comments (not the useful ones))
#############
device = 'cpu'  #torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load and preprocess the fat amount of data
# CSV FILE PATHS
#############
#CHANGE TO FILE PATHES YOU CAN KEEP EVERYTHING AFTER NLP BUT NOT NLP
##########################
train_csv_path = "/content/NLP-Project-Group-3/data/one_hot_targets/train_data.csv"  # Update if needed
test_csv_path  = "/content/NLP-Project-Group-3/data/one_hot_targets/test_data.csv"   # Update if needed

# marco del datos
df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

# this is just for something later i need a way to identify them and it cant just be
# the date bc some are same so it also need ticker and this was easiest
df_train["transcript_id"] = df_train["file_path"].apply(lambda x: os.path.basename(x))
df_test["transcript_id"] = df_test["file_path"].apply(lambda x: os.path.basename(x))


def load_transcript(file_path):
    ####################
    #CHANGE THE F STRING TO HAVE WHEREVER UR THING IS
    #KEEP EVERYTHING AFTER NLP BTU NOT NLP
    ##################
    with open(f'/content/NLP-Project-Group-3/{file_path}', "r", encoding="utf-8") as f:
        return f.read()

# preload the text bc the map function is trash and so is my load transcript but blame el mapo
# basically it stops it from constantly opening files and wasting everyones time
df_train["text"] = df_train["file_path"].apply(load_transcript)
df_test["text"] = df_test["file_path"].apply(load_transcript)

# rename for label
df_train = df_train.rename(columns={"combined_label": "labels"})
df_test  = df_test.rename(columns={"combined_label": "labels"})

# stupid hugging face gotta have they own stuff
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)


# Token and chunk section (must precompute before training)
# model name
#####################
#FIRST MODEL IS (aparently) BETTER BUT SECOND SEEMS TO RUN FASTER
#SEE HOW LONG FIRST FEW ITERATIONS TAKE AND DECIDE WHICH TO USE
############################################
model_name = "yiyanghkust/finbert-tone" # "yiyanghkust/finbert-tone" #"ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_chunk(batch):
    output_examples = {"input_ids": [], "attention_mask": [], "labels": [], "transcript_id": []}

    for text, label, transcript_id in zip(batch["text"], batch["labels"], batch["transcript_id"]):
        encoding = tokenizer(
            text,
            truncation=True,
            max_length=512,
            stride=128,
            return_overflowing_tokens=True,
            return_special_tokens_mask=False,
        )
        for input_ids, attention_mask in zip(encoding["input_ids"], encoding["attention_mask"]):
            output_examples["input_ids"].append(input_ids)
            output_examples["attention_mask"].append(attention_mask)
            output_examples["labels"].append(label)
            output_examples["transcript_id"].append(transcript_id)

    return output_examples

# hate these columns fr
columns_to_remove = ["file_path", "text", "surprise_pct", "volatility_change", "report_date", "ticker"]

# places to save the datasets so we dont need to reload everytime
###################
#CHANGE TO WHEREVER YOU WANT TO KEEP TEH TOKEN DATA WHEN RUNNING
###################################
train_dataset_path = "/content/gdrive/MyDrive/finbert-nlp/processed_train_dataset"
test_dataset_path  = "/content/gdrive/MyDrive/finbert-nlp/processed_test_dataset"

# processs + save
if not os.path.exists(train_dataset_path):
    dataset_train_chunked = dataset_train.map(
        tokenize_and_chunk,
        batched=True,
        batch_size=8,      # INCREASE THIS OR LOWER IT DEPENDING ON HOW LONG THINGS TAKE
        remove_columns=columns_to_remove,
        ##################################
        #INCREASE NUM_PROC IF U CAN JUST FIND OUT HOW MANY CPU CORES YOU GOT
        #####################
        num_proc=8,
        load_from_cache_file=False
    )
    # this part actually saves the stuff
    dataset_train_chunked.save_to_disk(train_dataset_path)
else:
    # this loads the stuff if it exists hopefully it does
    dataset_train_chunked = load_from_disk(train_dataset_path)

# same shit different story
if not os.path.exists(test_dataset_path):
    dataset_test_chunked = dataset_test.map(
        tokenize_and_chunk,
        batched=True,
        batch_size=8,       # INCREASE THIS OR LOWER IT DEPENDING ON HOW LONG THINGS TAKE
        remove_columns=columns_to_remove,
        ###########################
        #INCREASE NUM_PROC IF U CAN JUST FIND OUT HOW MANY CPU CORES YOU GOT
        #####################
        num_proc=8,
        load_from_cache_file=False
    )
    #saving stuff to file
    dataset_test_chunked.save_to_disk(test_dataset_path)
else:
    #loading stuff if it there
    dataset_test_chunked = load_from_disk(test_dataset_path)

# apartently we gotta set they own dataset class to be a certain way bc it bugs out otherwise
# it just makes the dataset be of pytorch tensors
dataset_train_chunked.set_format(type="torch", columns=["input_ids", "attention_mask", "labels", "transcript_id"])
dataset_test_chunked.set_format(type="torch", columns=["input_ids", "attention_mask", "labels", "transcript_id"])

counts = {
    lbl: dataset_train_chunked.filter(lambda ex, lbl=lbl: ex["labels"] == lbl).num_rows
    for lbl in range(4)
}
max_count = max(counts.values())

balanced_splits = []
print(counts)
for lbl, cnt in counts.items():
    # pull out all examples of this label
    ds_lbl = dataset_train_chunked.filter(lambda ex, lbl=lbl: ex["labels"] == lbl)
    if cnt < max_count:
        # repeat until we have at least max_count, then truncate
        reps = math.ceil(max_count / cnt)
        ds_lbl = concatenate_datasets([ds_lbl] * reps).select(range(max_count))
    balanced_splits.append(ds_lbl)
dataset_train_balanced = concatenate_datasets(balanced_splits).shuffle(seed=42)
# Training time

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,
    ignore_mismatched_sizes=True,
    )
model.to(device)

for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False

print("Trainable parameters:")
for n, p in model.named_parameters():
    if p.requires_grad:
        print("  ", n)
    else:
        print('not', n)
# train args
###################
#IF YOU CHANGED BATCH SIZE ABOVE I THINK DO SAME HERE
##################
training_args = TrainingArguments(
    output_dir="./finbert_chunked_output",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=5e-5,
)

# Define metric
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
# precision_metric = evaluate.load("precision")
# recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    # precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    # recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
        # "precision": precision["precision"],
        # "recall": recall["recall"]
    }

# I hate this trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_balanced,
    eval_dataset=dataset_test_chunked,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# TRAINING TIME BABY
trainer.train()
trainer.save_model("./final‑finetuned‑model")
tokenizer.save_pretrained("./final‑finetuned‑model")
# Evaluate lol i doubt well ever get to this part of the program
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

Filter:   0%|          | 0/32001 [00:00<?, ? examples/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fscbc (fscbc-boston-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.393700,1.370408,0.260206,0.183312
2,1.347900,1.340328,0.287422,0.210729
3,1.364900,1.373909,0.263148,0.212784
4,1.375400,1.329598,0.317580,0.244355
5,1.328100,1.346002,0.301949,0.236344
6,1.353100,1.312526,0.330452,0.238687
7,1.346200,1.329329,0.307466,0.235521
8,1.351000,1.330050,0.321626,0.243527
9,1.323000,1.317827,0.326407,0.239333
10,1.328300,1.324295,0.315189,0.234556


Evaluation results: {'eval_loss': 1.329598069190979, 'eval_accuracy': 0.31757999264435455, 'eval_f1': 0.2443545623701854, 'eval_runtime': 36.1118, 'eval_samples_per_second': 150.588, 'eval_steps_per_second': 18.83, 'epoch': 10.0}


In [ ]:
!zip -r /content/final-finetuned-model.zip /content/final‑finetuned‑model
!zip -r /content/finbert_chunked_output.zip /content/finbert_chunked_output
!zip -r /content/sample_data.zip /content/sample_data/
!zip -r /content/wandb.zip /content/wandb/

  adding: content/final‑finetuned‑model/ (stored 0%)
  adding: content/final‑finetuned‑model/model.safetensors (deflated 7%)
  adding: content/final‑finetuned‑model/tokenizer_config.json (deflated 74%)
  adding: content/final‑finetuned‑model/vocab.txt (deflated 50%)
  adding: content/final‑finetuned‑model/special_tokens_map.json (deflated 42%)
  adding: content/final‑finetuned‑model/config.json (deflated 53%)
  adding: content/final‑finetuned‑model/training_args.bin (deflated 51%)
  adding: content/final‑finetuned‑model/tokenizer.json (deflated 70%)
  adding: content/finbert_chunked_output/ (stored 0%)
  adding: content/finbert_chunked_output/checkpoint-6640/ (stored 0%)
  adding: content/finbert_chunked_output/checkpoint-6640/optimizer.pt (deflated 12%)
  adding: content/finbert_chunked_output/checkpoint-6640/model.safetensors (deflated 7%)
  adding: content/finbert_chunked_output/checkpoint-6640/tokenizer_config.json (deflated 74%)
  adding: content/finbert_chunked_output/checkpoint-

In [ ]:
from google.colab import files

files.download("/content/final-finetuned-model.zip")
files.download("/content/finbert_chunked_output.zip")
files.download("/content/sample_data.zip")
files.download("/content/wandb.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install safetensors

In [ ]:
!cp finbert_chunked_output.zip ./gdrive/MyDrive

In [ ]:
!cp final-finetuned-model.zip ./gdrive/MyDrive

# EVAL HERE

In [ ]:
import numpy as np
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from safetensors.torch import load_file
import pandas as pd
import os
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score

# Model and data loading
model_name = "yiyanghkust/finbert-tone"
model_path = "/content/gdrive/MyDrive/finbert-nlp/model.safetensors"
test_dataset_path = "/content/gdrive/MyDrive/finbert-nlp/processed_test_dataset"
test_csv_path = "/content/NLP-Project-Group-3/data/one_hot_targets/test_data.csv"

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state_dict = load_file(model_path)
model.classifier = torch.nn.Linear(model.classifier.in_features, 4)  # Update classifier
for key in state_dict:
    state_dict[key] = state_dict[key].to(device)
model.load_state_dict(state_dict, strict=False)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load test dataset
dataset_test_chunked = load_from_disk(test_dataset_path)
df_test = pd.read_csv(test_csv_path)

# Function to load the transcript text
def load_transcript(file_path):
    with open(f'/content/NLP-Project-Group-3/{file_path}', "r", encoding="utf-8") as f:
        return f.read()

# Create a dictionary mapping transcript_id to the full transcript text
transcript_texts = {os.path.basename(file_path): load_transcript(file_path) for file_path in df_test["file_path"]}

# Add the 'text' column back to the dataset
def add_text_column(example):
    example["text"] = transcript_texts[example["transcript_id"]]
    return example

dataset_test_chunked = dataset_test_chunked.map(add_text_column)

# Prediction function
def predict(text):
    encoding = tokenizer(text, truncation=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()
    return predictions[0]  # Return single prediction value

# Test functions
def test_first_512(example):
    return predict(example["text"][:512])

def test_last_512(example):
    return predict(example["text"][-512:])

def test_random_512(example):
    text = example["text"]
    start_idx = np.random.randint(0, len(text) - 512) if len(text) > 512 else 0
    return predict(text[start_idx:start_idx + 512])

def test_mean_pooling(example):
    chunks = [example["text"][i:i + 512] for i in range(0, len(example["text"]), 512)]

    # Batch the chunks (e.g., in batches of 32)
    batch_size = 32
    all_predictions = []  # Store predictions for all chunks

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i : i + batch_size]

        # Tokenize and get predictions for the batch
        encoding = tokenizer(batch, truncation=True, max_length=512, padding=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**encoding)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        all_predictions.extend(predictions)  # Add batch predictions to the list

    # Calculate mean after processing all chunks
    return int(round(np.mean(all_predictions)))


def test_max_pooling(example):
    chunks = [example["text"][i:i + 512] for i in range(0, len(example["text"]), 512)]

    # Batch the chunks (e.g., in batches of 32)
    batch_size = 32
    all_predictions = []  # Store predictions for all chunks

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i : i + batch_size]

        # Tokenize and get predictions for the batch
        encoding = tokenizer(batch, truncation=True, max_length=512, padding=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**encoding)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        all_predictions.extend(predictions)  # Add batch predictions to the list

    # Calculate max after processing all chunks
    return int(np.max(all_predictions))

# Run tests and store results
results = defaultdict(list)
for example in dataset_test_chunked:
    transcript_id = example["transcript_id"]
    results[transcript_id].append({
        "first_512": test_first_512(example),
        "last_512": test_last_512(example),
        "random_512": test_random_512(example),
        #"mean_pooling": test_mean_pooling(example),
        #"max_pooling": test_max_pooling(example),
    })

# Run tests and store results
all_predictions = defaultdict(list)
ground_truth_labels = []
for example in dataset_test_chunked:
    all_predictions["first_512"].append(test_first_512(example))
    all_predictions["last_512"].append(test_last_512(example))
    all_predictions["random_512"].append(test_random_512(example))
    all_predictions["mean_pooling"].append(test_mean_pooling(example))
    all_predictions["max_pooling"].append(test_max_pooling(example))
    ground_truth_labels.append(example["labels"])

# Calculate and print overall metrics
print("Overall Metrics for Test Dataset:")
for method in ["first_512", "last_512", "random_512", "mean_pooling", "max_pooling"]:
    predictions = all_predictions[method]
    accuracy = np.mean(np.array(predictions) == np.array(ground_truth_labels))
    precision = precision_score(ground_truth_labels, predictions, average='macro')
    recall = recall_score(ground_truth_labels, predictions, average='macro')
    f1 = f1_score(ground_truth_labels, predictions, average='macro')

    print(f"  {method}:")
    print(f"    Accuracy: {accuracy}")
    print(f"    Precision (macro): {precision}")
    print(f"    Recall (macro): {recall}")
    print(f"    F1-score (macro): {f1}")
    print("-" * 20)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-tone and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

Overall Metrics for Test Dataset:
  first_512:
    Accuracy: 0.42386906951084957
    Precision (macro): 0.10596726737771239
    Recall (macro): 0.25
    F1-score (macro): 0.1488441172672091
--------------------
  last_512:
    Accuracy: 0.09599117322545053
    Precision (macro): 0.023997793306362633
    Recall (macro): 0.25
    F1-score (macro): 0.043791946308724836
--------------------
  random_512:
    Accuracy: 0.2999264435454211
    Precision (macro): 0.2678029513559451
    Recall (macro): 0.27782904796735536
    F1-score (macro): 0.23578021186347659
--------------------
  mean_pooling:
    Accuracy: 0.4310408238322913
    Precision (macro): 0.1927049368684516
    Recall (macro): 0.2798834577144274
    F1-score (macro): 0.20749767160225424
--------------------
  max_pooling:
    Accuracy: 0.42386906951084957
    Precision (macro): 0.10596726737771239
    Recall (macro): 0.25
    F1-score (macro): 0.1488441172672091
--------------------


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Pool Eval

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, classification_report
import time
# CSV
csv_path = "/content/gdrive/MyDrive/finbert-nlp/test_data.csv"
df = pd.read_csv(csv_path)

# load full transcript
def load_transcript(fp):
    with open(f'/content/NLP-Project-Group-3/{fp}', "r", encoding="utf-8") as f:
        return f.read()
df["text"] = df["file_path"].apply(load_transcript)

# rename for HF
df = df.rename(columns={"combined_label": "labels"})

# file path is trans id
ds = Dataset.from_pandas(
    df[["text","labels","file_path"]],
    preserve_index=False
).rename_column("file_path","transcript_id")

# model
model_dir = "/content/gdrive/MyDrive/final-finbert-model/"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# no overlap
def tokenize_no_overlap(example):
    enc = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        stride=0,
        return_overflowing_tokens=True,
        return_attention_mask=True,
    )
    n = len(enc["input_ids"])
    # each segment has label bc stupid HF, i fix later
    return {
        "input_ids":      enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "labels":         [example["labels"]] * n,
        "transcript_id":  [example["transcript_id"]] * n,
    }

# map
ds_chunked = ds.map(
    tokenize_no_overlap,
    batched=False,
    remove_columns=["text"],
)

# format
ds_chunked.set_format(
    type="torch",
    columns=["input_ids","attention_mask","labels","transcript_id"]
)

print("Produced", len(ds_chunked), "chunks across", len(df), "documents.")


# does both
def evaluate_pooling(ds_docs):
    docs_cls = {}   # trans id: list of chunk CLS embeddings
    docs_lbl = {}
    start = time.perf_counter()
    k = 1
    with torch.no_grad():
        for ex in ds_docs:

            chunks_ids = ex["input_ids"]
            chunks_mask = ex["attention_mask"]
            tids = ex["transcript_id"]
            # each entry in labels is same
            lbl = ex["labels"][0].item() if isinstance(ex["labels"], torch.Tensor) else ex["labels"][0]

            # iterate each chunk
            for i in range(chunks_ids.size(0)):
                input_ids = chunks_ids[i].unsqueeze(0).to(device)
                attention_mask = chunks_mask[i].unsqueeze(0).to(device)
                bert_out = model.bert(input_ids=input_ids, attention_mask=attention_mask)
                cls_emb = bert_out.pooler_output.squeeze(0).cpu()
                tid = tids[i]
                docs_cls.setdefault(tid, []).append(cls_emb)
                docs_lbl.setdefault(tid, lbl)
            print(f'{k}: {time.perf_counter()-start}')
            k+=1

    # pool per document and classify
    y_true, y_pred_mean, y_pred_max = [], [], []
    for tid, embs in docs_cls.items():
        stacked = torch.stack(embs, dim=0)
        if k == 101:
            print(stacked.shape)
            k+= 1
        # mean pool
        doc_emb_mean = stacked.mean(dim=0)
        doc_emb_mean = doc_emb_mean.unsqueeze(0).to(device)
        logits_mean = model.classifier(doc_emb_mean)
        pred_mean = logits_mean.argmax(dim=-1).item()
        y_pred_mean.append(pred_mean)
         # max pool
        doc_emb_max, _ = stacked.max(dim=0)
        doc_emb_max = doc_emb_max.unsqueeze(0).to(device)
        logits_max = model.classifier(doc_emb_max)
        pred_max = logits_max.argmax(dim=-1).item()
        y_pred_max.append(pred_max)

        y_true.append(docs_lbl[tid])
    # Mean pool
    acc = accuracy_score(y_true, y_pred_mean)
    f1 = f1_score(y_true, y_pred_mean, average="macro")
    print(f"[MEAN POOL] Acc: {acc:.4f}, Macro-F1: {f1:.4f}")
    print(classification_report(y_true, y_pred_mean, zero_division=0))
    # Max pool
    acc = accuracy_score(y_true, y_pred_max)
    f1 = f1_score(y_true, y_pred_max, average="macro")
    print(f"[MAX POOL] Acc: {acc:.4f}, Macro-F1: {f1:.4f}")
    print(classification_report(y_true, y_pred_max, zero_division=0))

    # Per-dimension (ES / VC) metrics
    # mapping: 0->(0,0), 1->(0,1), 2->(1,0), 3->(1,1)
    es_true = [lbl//2 for lbl in y_true]
    es_pred_mean = [p//2 for p in y_pred_mean]
    es_pred_max = [p//2 for p in y_pred_max]

    vc_true = [lbl%2 for lbl in y_true]
    vc_pred_mean = [p%2 for p in y_pred_mean]
    vc_pred_max = [p%2 for p in y_pred_max]

    print(f"\n MEAN POOL")
    print(f"\n ES performance:")
    print(classification_report(es_true, es_pred_mean, labels=[0,1],
                                target_names=["ES=0","ES=1"], zero_division=0))
    print(f"\n VC performance:")
    print(classification_report(vc_true, vc_pred_mean, labels=[0,1],
                                target_names=["VC=0","VC=1"], zero_division=0))
    print(f"\n MAX POOL")
    print(f"\n ES performance:")
    print(classification_report(es_true, es_pred_max, labels=[0,1],
                                target_names=["ES=0","ES=1"], zero_division=0))
    print(f"\n VC performance:")
    print(classification_report(vc_true, vc_pred_max, labels=[0,1],
                                target_names=["VC=0","VC=1"], zero_division=0))

evaluate_pooling(ds_chunked)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Produced 100 chunks across 100 documents.
1: 0.2397863270000471
2: 0.4312397429999919
3: 0.6226877799999784
4: 0.8245631319999802
5: 1.024527175000003
6: 1.2240431700000727
7: 1.4063769580000098
8: 1.5969943790000798
9: 1.7958800600000586
10: 1.9951174820000688
11: 2.2036341900000025
12: 2.43773115099998
13: 2.6546461660000205
14: 2.88204994299997
15: 3.1083775560000504
16: 3.3250924050000776
17: 3.5431123790000356
18: 3.7611686840000402
19: 3.9603488750000224
20: 4.185963900000047
21: 4.325452250000012
22: 4.447730265000018
23: 4.622239193000041
24: 4.796127539000054
25: 4.969947397000055
26: 5.169782328999986
27: 5.369549330000041
28: 5.569512712999995
29: 5.751541780000025
30: 5.926762285999985
31: 6.1697890010000265
32: 6.404773349000038
33: 6.65639756500002
34: 6.874575148000076
35: 7.109104120999973
36: 7.336734641000021
37: 7.5908479460000535
38: 7.825030449999986
39: 8.050435880999999
40: 8.26710003200003
41: 8.52088010600005
42: 8.738175705999993
43: 8.973428845000058
44: 9.15

In [ ]:
# DEVICE SETUP
######################
#CHANGE TO COMMENTED LINE POR FAVOR (also delete these ass comments (not the useful ones))
#############
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load and preprocess the fat amount of data
# CSV FILE PATHS
#############
#CHANGE TO FILE PATHES YOU CAN KEEP EVERYTHING AFTER NLP BUT NOT NLP
##########################
train_csv_path = "/content/NLP-Project-Group-3/data/one_hot_targets/train_data.csv"  # Update if needed
test_csv_path  = "/content/NLP-Project-Group-3/data/one_hot_targets/test_data.csv"   # Update if needed

# marco del datos
df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

# this is just for something later i need a way to identify them and it cant just be
# the date bc some are same so it also need ticker and this was easiest
df_train["transcript_id"] = df_train["file_path"].apply(lambda x: os.path.basename(x))
df_test["transcript_id"] = df_test["file_path"].apply(lambda x: os.path.basename(x))


def load_transcript(file_path):
    ####################
    #CHANGE THE F STRING TO HAVE WHEREVER UR THING IS
    #KEEP EVERYTHING AFTER NLP BTU NOT NLP
    ##################
    with open(f'/content/NLP-Project-Group-3/{file_path}', "r", encoding="utf-8") as f:
        return f.read()

# preload the text bc the map function is trash and so is my load transcript but blame el mapo
# basically it stops it from constantly opening files and wasting everyones time
df_train["text"] = df_train["file_path"].apply(load_transcript)
df_test["text"] = df_test["file_path"].apply(load_transcript)

# rename for label
df_train = df_train.rename(columns={"combined_label": "labels"})
df_test  = df_test.rename(columns={"combined_label": "labels"})

# stupid hugging face gotta have they own stuff
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)


# Token and chunk section (must precompute before training)
# model name
#####################
#FIRST MODEL IS (aparently) BETTER BUT SECOND SEEMS TO RUN FASTER
#SEE HOW LONG FIRST FEW ITERATIONS TAKE AND DECIDE WHICH TO USE
############################################
model_name = "yiyanghkust/finbert-tone" # "yiyanghkust/finbert-tone" #"ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_chunk(batch):
    output_examples = {"input_ids": [], "attention_mask": [], "labels": [], "transcript_id": []}

    for text, label, transcript_id in zip(batch["text"], batch["labels"], batch["transcript_id"]):
        encoding = tokenizer(
            text,
            truncation=True,
            max_length=512,
            stride=128,
            return_overflowing_tokens=True,
            return_special_tokens_mask=False,
        )
        for input_ids, attention_mask in zip(encoding["input_ids"], encoding["attention_mask"]):
            output_examples["input_ids"].append(input_ids)
            output_examples["attention_mask"].append(attention_mask)
            output_examples["labels"].append(label)
            output_examples["transcript_id"].append(transcript_id)

    return output_examples

# hate these columns fr
columns_to_remove = ["file_path", "text", "surprise_pct", "volatility_change", "report_date", "ticker"]

# places to save the datasets so we dont need to reload everytime
###################
#CHANGE TO WHEREVER YOU WANT TO KEEP TEH TOKEN DATA WHEN RUNNING
###################################
train_dataset_path = "/content/gdrive/MyDrive/finbert-nlp/processed_train_dataset"
test_dataset_path  = "/content/gdrive/MyDrive/finbert-nlp/processed_test_dataset"

# processs + save
if not os.path.exists(train_dataset_path):
    dataset_train_chunked = dataset_train.map(
        tokenize_and_chunk,
        batched=True,
        batch_size=8,      # INCREASE THIS OR LOWER IT DEPENDING ON HOW LONG THINGS TAKE
        remove_columns=columns_to_remove,
        ##################################
        #INCREASE NUM_PROC IF U CAN JUST FIND OUT HOW MANY CPU CORES YOU GOT
        #####################
        num_proc=8,
        load_from_cache_file=True
    )
    # this part actually saves the stuff
    dataset_train_chunked.save_to_disk(train_dataset_path)
else:
    # this loads the stuff if it exists hopefully it does
    dataset_train_chunked = load_from_disk(train_dataset_path)

# same shit different story
if not os.path.exists(test_dataset_path):
    dataset_test_chunked = dataset_test.map(
        tokenize_and_chunk,
        batched=True,
        batch_size=8,       # INCREASE THIS OR LOWER IT DEPENDING ON HOW LONG THINGS TAKE
        remove_columns=columns_to_remove,
        ###########################
        #INCREASE NUM_PROC IF U CAN JUST FIND OUT HOW MANY CPU CORES YOU GOT
        #####################
        num_proc=8,
        load_from_cache_file=True
    )
    #saving stuff to file
    dataset_test_chunked.save_to_disk(test_dataset_path)
else:
    #loading stuff if it there
    dataset_test_chunked = load_from_disk(test_dataset_path)

# apartently we gotta set they own dataset class to be a certain way bc it bugs out otherwise
# it just makes the dataset be of pytorch tensors
dataset_train_chunked.set_format(type="torch", columns=["input_ids", "attention_mask", "labels", "transcript_id"])
dataset_test_chunked.set_format(type="torch", columns=["input_ids", "attention_mask", "labels", "transcript_id"])


# Training time

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)
model.to(device)

# train args
###################
#IF YOU CHANGED BATCH SIZE ABOVE I THINK DO SAME HERE
##################
training_args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/finbert-nlp",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Define metric
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
# precision_metric = evaluate.load("precision")
# recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    # precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    # recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
        # "precision": precision["precision"],
        # "recall": recall["recall"]
    }

# I hate this trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_chunked,
    eval_dataset=dataset_test_chunked,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# TRAINING TIME BABY
trainer.train()

# Evaluate lol i doubt well ever get to this part of the program
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-tone and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<ipython-input-11-c9970c6d4847>:175: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyboardInterrupt: 

In [ ]:
import torch
from safetensors.torch import load_file
from transformers import AutoModelForSequenceClassification

model_name = "yiyanghkust/finbert-tone"
model_path = "/content/gdrive/MyDrive/finbert-nlp/model.safetensors"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)  # Load with original num_labels=3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state_dict = load_file(model_path)

# Update model's classifier layer to match the number of labels in your training data (4 in this case)
model.classifier = torch.nn.Linear(model.classifier.in_features, 4)

for key in state_dict:
    state_dict[key] = state_dict[key].to(device)
model.load_state_dict(state_dict, strict=False)  # Load with strict=False to ignore the classifier layer mismatch
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-tone and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Load test dataset and relevant files
test_dataset_path = "/content/gdrive/MyDrive/finbert-nlp/processed_test_dataset"  # Update if needed
test_csv_path  = "/content/NLP-Project-Group-3/data/one_hot_targets/test_data.csv"   # Update if needed
dataset_test_chunked = load_from_disk(test_dataset_path)
df_test = pd.read_csv(test_csv_path)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to load the transcript text
def load_transcript(file_path):
    with open(f'/content/NLP-Project-Group-3/{file_path}', "r", encoding="utf-8") as f:
        return f.read()

# Create a dictionary mapping transcript_id to the full transcript text
transcript_texts = {os.path.basename(file_path): load_transcript(file_path) for file_path in df_test["file_path"]}

# Add the 'text' column back to the dataset
def add_text_column(example):
    example["text"] = transcript_texts[example["transcript_id"]]
    return example

dataset_test_chunked = dataset_test_chunked.map(add_text_column)

# Prediction function
def predict(text):
    encoding = tokenizer(text, truncation=True, max_length=512, return_tensors="pt")
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()
    return predictions

# Test functions (1-3 & random)
def test_first_512(dataset):
    results = []
    for example in dataset:
        text = example["text"][:512]
        prediction = predict(text)
        results.append(prediction)
    return results

def test_second_512(dataset):
    results = []
    for example in dataset:
        text = example["text"][512:1024]
        prediction = predict(text)
        results.append(prediction)
    return results


def test_last_512(dataset):
    results = []
    for example in dataset:
        text = example["text"][-512:]
        prediction = predict(text)
        results.append(prediction)
    return results

def test_random_512(dataset):
    results = []
    for example in dataset:
        text = example["text"]
        if len(text) <= 512:  # Handle cases where text is shorter than 512 tokens
            start_idx = 0
        else:
            start_idx = np.random.randint(0, len(text) - 512)
        text_chunk = text[start_idx : start_idx + 512]
        prediction = predict(text_chunk)
        results.append(prediction)
    return results

# Test functions (4-5 mean/max pooling)
def test_mean_pooling(dataset):
    results = []
    for example in dataset:
        text = example["text"]
        chunks = [text[i : i + 512] for i in range(0, len(text), 512)]
        chunk_predictions = [predict(chunk) for chunk in chunks]
        mean_prediction = np.mean(chunk_predictions, axis=0)
        results.append(mean_prediction)
    return results

def test_max_pooling(dataset):
    results = []
    for example in dataset:
        text = example["text"]
        chunks = [text[i : i + 512] for i in range(0, len(text), 512)]
        chunk_predictions = [predict(chunk) for chunk in chunks]
        max_prediction = np.max(chunk_predictions, axis=0)
        results.append(max_prediction)
    return results

In [ ]:
# Load the test dataset
test_dataset = dataset_test_chunked

# Run the tests and print the results
first_512_results = test_first_512(test_dataset)
second_512_results = test_second_512(test_dataset)
#last_512_results = test_last_512(test_dataset)
#random_512_results = test_random_512(test_dataset)

print("First 512 tokens results:", first_512_results)
print("Second 512 tokens results:", second_512_results)
#print("Last 512 tokens results:", last_512_results)
#print("Random 512 tokens results:", random_512_results)


First 512 tokens results: [array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), array([3]), a

In [ ]:
mean_pooling_results = test_mean_pooling(test_dataset)
max_pooling_results = test_max_pooling(test_dataset)


In [ ]:
for method in ["first_512", "last_512", "random_512"]:#, "mean_pooling", "max_pooling"]:
    predictions = all_predictions[method]
    accuracy = np.mean(np.array(predictions) == np.array(ground_truth_labels))
    precision = precision_score(ground_truth_labels, predictions, average='macro')
    recall = recall_score(ground_truth_labels, predictions, average='macro')
    f1 = f1_score(ground_truth_labels, predictions, average='macro')

    print(f"  {method}:")
    print(f"    Accuracy: {accuracy}")
    print(f"    Precision (macro): {precision}")
    print(f"    Recall (macro): {recall}")
    print(f"    F1-score (macro): {f1}")
    print("-" * 20)

NameError: name 'all_predictions' is not defined

In [ ]:
for method in ["mean_pooling", "max_pooling"]:
    predictions = all_predictions[method]
    accuracy = np.mean(np.array(predictions) == np.array(ground_truth_labels))
    precision = precision_score(ground_truth_labels, predictions, average='macro')
    recall = recall_score(ground_truth_labels, predictions, average='macro')
    f1 = f1_score(ground_truth_labels, predictions, average='macro')

    print(f"  {method}:")
    print(f"    Accuracy: {accuracy}")
    print(f"    Precision (macro): {precision}")
    print(f"    Recall (macro): {recall}")
    print(f"    F1-score (macro): {f1}")
    print("-" * 20)

NameError: name 'all_predictions' is not defined

In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from safetensors.torch import load_file
import pandas as pd
import os

# Model and data loading
model_name = "yiyanghkust/finbert-tone"
model_path = "/content/gdrive/MyDrive/finbert-nlp/model.safetensors"
test_csv_path = "/content/NLP-Project-Group-3/data/one_hot_targets/test_data.csv"

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4, ignore_mismatched_sizes=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state_dict = load_file(model_path)
model.classifier = torch.nn.Linear(model.classifier.in_features, 4)  # Update classifier
for key in state_dict:
    state_dict[key] = state_dict[key].to(device)
model.load_state_dict(state_dict, strict=False)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load test data
df_test = pd.read_csv(test_csv_path)

# Function to load the transcript text
def load_transcript(file_path):
    with open(f'/content/NLP-Project-Group-3/{file_path}', "r", encoding="utf-8") as f:
        return f.read()

# Create a dictionary mapping transcript_id to the full transcript text
transcript_texts = {os.path.basename(file_path): load_transcript(file_path) for file_path in df_test["file_path"]}

# Add the 'text' column back to the dataset
df_test["text"] = df_test["file_path"].apply(lambda x: transcript_texts[os.path.basename(x)])

# Prediction function
def predict(text):
    encoding = tokenizer(text, truncation=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()
    return predictions[0], logits.cpu().numpy()  # Return prediction and logits

# Manually inspect a few predictions
for i in range(8):  # Inspect first 5 examples
    text = df_test.iloc[i]["text"]
    true_label = df_test.iloc[i]["combined_label"]
    prediction, logits = predict(text[:512])  # First 512 tokens
    print(f"True label: {true_label}, Prediction: {prediction}, Logits: {logits}")

    prediction, logits = predict(text[-512:])  # Last 512 tokens
    print(f"True label: {true_label}, Prediction: {prediction}, Logits: {logits}")

    start_idx = np.random.randint(0, len(text) - 512) if len(text) > 512 else 0
    prediction, logits = predict(text[start_idx:start_idx + 512])  # Random 512 tokens
    print(f"True label: {true_label}, Prediction: {prediction}, Logits: {logits}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-tone and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True label: 2, Prediction: 3, Logits: [[-0.38738346 -0.8231265   0.80022335  0.93800914]]
True label: 2, Prediction: 3, Logits: [[-0.38737643 -0.8231082   0.80021566  0.93801254]]
True label: 2, Prediction: 3, Logits: [[-0.3873882 -0.823143   0.8002309  0.93801  ]]
True label: 3, Prediction: 3, Logits: [[-0.38738346 -0.82312673  0.8002235   0.93800926]]
True label: 3, Prediction: 3, Logits: [[-0.38737583 -0.8231068   0.8002153   0.9380127 ]]
True label: 3, Prediction: 3, Logits: [[-0.38739106 -0.82314974  0.8002343   0.9380095 ]]
True label: 1, Prediction: 3, Logits: [[-0.38738197 -0.8231224   0.8002215   0.93800944]]
True label: 1, Prediction: 3, Logits: [[-0.38737583 -0.8231068   0.8002153   0.9380127 ]]
True label: 1, Prediction: 3, Logits: [[-0.3873898  -0.8231458   0.8002319   0.93801033]]
True label: 2, Prediction: 3, Logits: [[-0.38738126 -0.82312053  0.80022067  0.9380096 ]]
True label: 2, Prediction: 3, Logits: [[-0.38737583 -0.8231068   0.8002153   0.9380127 ]]
True label: 2,